[View in Colaboratory](https://colab.research.google.com/github/praveen460/datascience/blob/master/mnist_cnn_2.ipynb)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("mnist/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models

In [0]:
x=tf.placeholder(tf.float32,shape=[None,784])
y_true=tf.placeholder(tf.float32,shape=[None,10])

In [0]:
def init_weights(shape):
  init_weights_cal=tf.truncated_normal(shape=shape,stddev=0.1)
  return (tf.Variable(init_weights_cal))


In [0]:
def init_bias(shape):
  init_bias_cal=tf.constant(0.1,shape=shape)
  return(tf.Variable(init_bias_cal))

In [0]:
def conv2d(x,w):
  return(tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME'))

In [0]:
def max_pool(x):
  return(tf.nn.max_pool(x,strides=[1,2,2,1],padding='SAME',ksize=[1,2,2,1]))

In [0]:
def conv_layer(x,shape):
  w=init_weights(shape)
  b=init_bias([shape[3]])
  return(tf.nn.relu(conv2d(x,w)+b))

In [0]:
def fcc_layer(x,size):
  input_size=int(x.get_shape()[1])
  w=init_weights([input_size,size])
  b=init_bias([size])
  return(tf.matmul(x,w)+b)
  

In [0]:
x_image=tf.reshape(x,[-1,28,28,1])

In [0]:
conv_1=conv_layer(x_image,shape=[10,10,1,64])

In [0]:
pool_1=max_pool(conv_1)

In [0]:
conv_2=conv_layer(pool_1,shape=[10,10,64,128])

In [0]:
pool_2=max_pool(conv_2)

In [0]:
conv_flat=tf.reshape(pool_2,shape=[-1,7*7*128])

In [0]:
fcc1=tf.nn.relu(fcc_layer(conv_flat,1024))

In [0]:
hold_prob=tf.placeholder(tf.float32)

In [0]:
full_one_dropout = tf.nn.dropout(fcc1,keep_prob=hold_prob)
y_pred=fcc_layer(full_one_dropout,10)

In [20]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [0]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)

In [0]:
train=optimizer.minimize(loss)

In [0]:
saver=tf.train.Saver()

In [0]:
init=tf.global_variables_initializer()

In [25]:
with tf.Session() as sess:
  sess.run(init)
  for i in range(10000):
    batch_x,batch_y=mnist.train.next_batch(100)
    sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:1.0})
  predictions=tf.argmax(y_pred,1)
  saved_path=saver.save(sess,"/tmp/cnn_2.ckpt")
  print("saved_path = {}".format(saved_path))

saved_path = /tmp/cnn_2.ckpt


In [26]:
with tf.Session() as sess:
  pred=[]
  y_test=[]
  saver.restore(sess,"/tmp/cnn_2.ckpt")
  for i in range(10000):
    test_x,test_y=mnist.test.next_batch(1)
    prediction=sess.run(predictions,feed_dict={x:test_x,hold_prob:1.0})
    pred.append(prediction[0])
    y_test.append(np.nonzero(test_y)[1])
    #print ("Predicted = {} , actual = {}".format(prediction,y_test))
    #print(y_test)

INFO:tensorflow:Restoring parameters from /tmp/cnn_2.ckpt


In [27]:
count=0
for i in range(10000):
  if y_test[i]==pred[i]:
    count+=1
print("accuracy={}".format(count*100/10000))

accuracy=96.66
